# Demonstration of spherical convolution

In this small notebook, we test an implementation of a spherical convolution. The general idea is to use a graph instead of the tradtionial 2 dimensional grid as a support for convolution.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pygsp
pygsp.plotting.BACKEND = 'matplotlib'
from pprint import pprint
import math
import scipy
import itertools

from scnn import utils, models

%matplotlib notebook



## Definition of the graph

Let us start by defining the graph on the healpix sampling scheme.

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
NSIDE = 8
NPIX = NSIDE**2*12
[x,y,z] = hp.pix2vec(NSIDE, range(NPIX),nest=True)
ax.scatter(x,y,z)

We use a 8 nearest neighboor graph

In [ ]:
# # build a healpix graph from NSIDE
# def healpix_graph(nside=16, nest=False, lap_type='normalized'):
#     NPIX = nside**2*12 # number of pixels
#     pix = range(NPIX)
    
#     # 1) get the coordinates
#     [x,y,z] = hp.pix2vec(nside, pix,nest=nest)
#     coords = np.vstack([x,y,z]).transpose()
    
#     # 2) get the 8 neighboors
#     [theta, phi] = hp.pix2ang(nside, pix, nest = nest)
#     index_neighboor = hp.pixelfunc.get_all_neighbours(nside,theta=theta, phi=phi, nest = nest)
    
#     # 3) build the adjacency matrix
# #     row_index = []
# #     col_index = []
# #     for row in pix:
# #         for col in index_neighboor[:,row]:
# #             if col>=0:
# #                 row_index.append(row)
# #                 col_index.append(col)
#     col_index = np.reshape(index_neighboor.T,[NPIX*8])
#     row_index = np.reshape(np.reshape(np.array(list(pix)*8),[8,NPIX]).T,[8*NPIX])
#     good_index = col_index >= 0
#     col_index = col_index[good_index]
#     row_index = row_index[good_index]
    
#     # index_sparse = [(row,col) for row in range(len(ind)) for col in index_neighboor[:,row]]
#     dist = np.array([sum((coords[row]-coords[col])**2) for row,col in zip(row_index,col_index)])
#     mean_dist = np.mean(dist)
#     w = np.exp(-dist/(2*mean_dist))
#     W = scipy.sparse.csr_matrix((w,(row_index, col_index)), shape=(NPIX,NPIX))
    
#     G = pygsp.graphs.Graph(W, gtype='healpix', lap_type=lap_type, coords=coords)
#     return G
    

We start with a small graph

In [ ]:
G = utils.healpix_graph(nside=16,lap_type='normalized', nest = True, dtype=np.float64)
print('max degree: {}'.format(np.max(np.sum(G.W,axis=0))))
print('min degree: {}'.format(np.min(np.sum(G.W,axis=0))))
print('mean degree: {}'.format(np.mean(np.sum(G.W,axis=0))))
print('Is the graph directed? {}'.format(G.is_directed()))
print('Number of nodes: {}'.format(G.N))

## Analysis of the graph

Graph convolution is defined as the pointwise multiplication in the graph spectral domain. Hence it is important to veryfy the spectral property of the graph. Note that this operation requires the diagonalization of the Laplacian, which is very costly in computations and mermory. Nevertheless, when it comes to convolution, their exist fast methods that only require sparse matrix multiplications.

The eigenvectors are obtained by diagonalizing the graph laplacian defined as $L=I-D^{\frac{1}{2}}WD^{\frac{1}{2}}$, where $W$ is the weight/adjacency matrix and $D$ the degree matrix. 

The Fourier basis $U$ by definition satisfies
$$ L  = U \Lambda U^*. $$
Here the eigenvalues contained in the diagonal of $\Lambda$ somehow correspond to the graph squared frequencies.

In [ ]:
# Compute all eigenvectors
G.compute_fourier_basis()

In [ ]:
Uv = np.reshape(G.U,[G.N**2,1])
print('Mean: {}'.format(np.mean(np.abs(Uv))))
print('Min: {}'.format(np.min(Uv)))
print('Max: {}'.format(np.max(Uv)))
print('Perline: {}'.format(np.max(G.U,axis=1)))

Let us display a few Fourier modes on the healpix map.

In [ ]:
fig = plt.figure(figsize=(8, 4))
ne = 16

for ind in range(ne):
    hp.mollview(G.U[:,ind], 
                title="Eigenvector {}".format(ind), 
                nest=True, 
                sub=(math.sqrt(ne),math.sqrt(ne),ind+1),
                max=np.max(np.abs(G.U[:,:ne])),
                min=-np.max(np.abs(G.U[:,:ne])),
                cbar=False)


We should also check higher frequency modes as they can be more localized.

In [ ]:
ind = 3000
fig = plt.figure(figsize=(3, 2))
hp.mollview(G.U[:,ind], title="Eigenvector {}".format(ind), nest=True, cbar=False, sub=(1,1,1))

The most localized eigenvector is considered to be the one with the heighest coherence.

In [ ]:
ind = np.argmax(np.max(np.abs(G.U),axis=0))
fig = plt.figure(figsize=(3, 2))
hp.mollview(G.U[:,ind], title="Eigenvector {}".format(ind), nest=True, cbar=False, sub=(1,1,1))

This eigenvector is clearly very localized. Let us display the modulus of the Fourier eigenvector to have a more general idea about all eigenvectors.

In [ ]:
fig = plt.figure()
plt.imshow(np.abs(G.U), cmap='Greys')

## Effect of the convolution

The convolution of a signal $f$ and a kernel $k(x)$ on a graph is defined as the pointwise multiplication in the spectral domain, i.e.
$$f_c  = U k(\Lambda)U^*f. $$
Here $U^*f$ is the graph Fourier transform of $f$ and $k(\Lambda)$ is a diagonal matrix where the kernel $k$ applied on each element of the diagonal of $\Lambda$. 

Let us start with the heat diffusion problem. We solve the following equation on the graph:
$$ L f(t) = \tau \partial_t f(t),$$
where $f(t): \mathbb{R}_+ \rightarrow \mathbb{R}^N$ is a multivariate function depending on the time, $L$ a positive semi-definite matrix representing the Laplacian on a graph, and $\tau$ a constant.

Given the vector $f_0 = f(0)$, the solution of this equation for time $t$ can be written as:
$$ f(t) = K_t(L) f_0, $$
where 
$$ K_t(L) = e^{-\tau t L}.$$
In the equation $f(t) = K_t(L) f_0$, the kernel $K_t(x)=e^{-\tau t x}$ can be consdired as the convolution kernel and the heat diffusion problem can be solved using a simple convolution on the graph.

In [ ]:
tau = [5,10,20,50]
hf = pygsp.filters.Heat(G, tau=tau)
fig, ax = plt.subplots()
hf.plot(plot_eigenvalues=True, show_sum=False, ax=ax)
_ = ax.set_title('Filter frequency response')

In [ ]:
ind0 = 0
sig = np.zeros([G.N])
sig[ind0]=1
conv = hf.analyze(sig)
fig = plt.figure(figsize=(6, 3))
ne = 4

for ind in range(ne):
    hp.mollview(conv[:,ind], 
                title="Tau: {}".format(tau[ind]), 
                nest=True, 
                sub=(2,2,ind+1))

ind0 = 500
sig = np.zeros([G.N])
sig[ind0]=1
conv = hf.analyze(sig)
fig = plt.figure(figsize=(6, 3))
ne = 4

for ind in range(ne):
    hp.mollview(conv[:,ind], 
                title="Tau: {}".format(tau[ind]), 
                nest=True, 
                sub=(2,2,ind+1))

## Testing using Planck map

Let us play with with a Planck map.

In [ ]:
map_cmb, map_noise, map_mask = hp.read_map('data/COM_CMB_IQU-smica_1024_R2.02_full.fits', field = (0,1,3), nest=True)

In [ ]:
hp.mollview(map_cmb, title='cmb', nest=True)
# hp.mollview(map_noise, title='noise', cmap='RdBu')
# hp.mollview(map_mask, title='mask', cmap='RdBu')

Let us first select a lower resolution: NSIDE=256, making the total number of pixels: $ N = 256^2*12=786432.$

In [ ]:
nside = 256
map_cmb_lores = hp.ud_grade(map_cmb, nside_out=nside, order_in='NESTED')
G = utils.healpix_graph(nside=nside, nest = True)
G.estimate_lmax()

Let apply our heat operator. It will smooth the map.

In [ ]:
tau = [5,10,20,50]
hf = pygsp.filters.Heat(G, tau=tau)

In [ ]:
conv_map_lowres = hf.analyze(map_cmb_lores)
fig = plt.figure(figsize=(10, 5))
ne = 4

for ind in range(ne):
    hp.mollview(conv_map_lowres[:,ind], 
                title="Tau: {}".format(tau[ind]), 
                nest=True, 
                sub=(2,2,ind+1))

Let us now compute the power spectral density on the sphere. This is going to be different than the traditionial one.

In [ ]:
def estimate_graph_psd(G, sig, Nrand=10, Npoint=30):
    from scipy.interpolate import interp1d
    g = pygsp.filters.Itersine(G, Nf=Npoint, overlap=2)
    mu = np.linspace(0,G.lmax,Npoint)

    sig_filt = g.filter(sig, method='chebyshev', order=2*Npoint)
#     sig_dist = np.mean(np.sum(sig_filt*sig_filt,axis=0),axis=0).squeeze()
    sig_dist = np.sum(sig_filt*sig_filt,axis=0)
    if len(sig_dist.shape)>1:
        sig_dist = np.mean(sig_dist,axis=0).squeeze()
    
    rand_sig = np.random.binomial(n=1,p=0.5, size=[G.N,Nrand])*2 - 1
    rand_sig_filered = g.filter(rand_sig, method='chebyshev', order=2*Npoint )
    eig_dist = np.mean(np.sum(rand_sig_filered*rand_sig_filered,axis=0),axis=0).squeeze()
    
    psd_values = sig_dist/eig_dist
    inter = interp1d(mu, psd_values, kind='linear')
    return pygsp.filters.Filter(G, inter), (mu,psd_values)

psd_filter, psd_point = estimate_graph_psd(G, map_cmb_lores, Nrand=5, Npoint=30)
    
    
        

In [ ]:
psd_filter.plot()
plt.plot(*psd_point, 'x')

In [ ]:
plt.figure()
plt.semilogy(*psd_point)